### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import random
import math

from sklearn import preprocessing # normalization library


### Robot Data
Pioneer 3dx

In [2]:

LMr=381e-3
R=195e-3

v_max= 1
v_min=0.5

Lt_max=1.2
Lt_min=0.3

### Obstacle circle 

Sensor range data: $[dmin_1\quad dmax_1]\quad m$ 

Heading angle data: $[-\pi\quad\pi]\quad rad$ 


In [3]:
min_d1=0.3
max_d1=1

In [4]:
# Sensor Data gen 
Obs_Data=[round(random.uniform(min_d1, max_d1),4) for _ in range(3000*3)]
Obs_Data_M=np.array(Obs_Data).reshape(3000,3)

print('Raw data :',Obs_Data_M[:2,:])

normalizer=preprocessing.MinMaxScaler()
Obs_Data_Norm=normalizer.fit_transform(Obs_Data_M)
print('Normalized data :',Obs_Data_Norm[:2,:])


Raw data : [[0.924  0.5576 0.5139]
 [0.5983 0.4992 0.9707]]
Normalized data : [[0.89152494 0.369319   0.3057662 ]
 [0.42603973 0.2855914  0.95936472]]


In [5]:
# HA data gen

HA_Data=[round(random.uniform(-math.pi, math.pi),4) for _ in range(3000)]
HA_Data=np.array(HA_Data).reshape(3000,1)
print('Angle data :',HA_Data[:2])

Angle data : [[-0.8852]
 [ 0.6788]]


In [6]:
# Stack All

St_train_C1=np.hstack((Obs_Data_Norm,HA_Data))

Df_C1=pd.DataFrame(St_train_C1,columns=['LD','FD','RD','HA'])

Df_C1.sample(6)

,LD,FD,RD,HA
1119,0.507217,0.225090,0.986980,2.3684
1134,0.248678,0.760287,0.162255,1.7343
1014,0.147921,0.512545,0.916869,-2.6697
2131,0.755181,0.317993,0.508370,-0.7168
253,0.156353,0.494194,0.353126,-0.2052
2167,0.026154,0.556416,0.158678,-1.7309


### Inverse Kinematic Model

$$\omega_{R/L}=\left( 1\pm\frac{L_{MR}}{2L_T}\right)\frac{v}{R}$$

Fist Case: Left sensor nearest, objective angle +

In [7]:
odp=1
tethad=3
S=-1
Lt_p=round((odp*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(((odp)*(v_max-v_min)+v_min),4)
print('vel: ',V_p)
W_R=round((1+((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

Ltp:  1.2
vel:  1.0
Right Wheel vel:  2.6859
Left Wheel vel:  7.5705


Second Case: Left sensor nearest, objective angle -

In [8]:

odp=0.1
tethad=-3
S=0.5
Lt_p=round((odp*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(((odp)*(v_max-v_min)+v_min),4)
print('vel: ',V_p)
W_R=round((1+((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

Ltp:  0.39
vel:  0.55
Right Wheel vel:  0.7539
Left Wheel vel:  4.8871


Third case: Right sensor nearest, objective angle +

In [9]:
odp=0.114049
tethad=0.3
S=0.5
Lt_p=round((odp*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(((odp)*(v_max-v_min)+v_min),4)
print('vel: ',V_p)
W_R=round((1+((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

Ltp:  0.4026
vel:  0.557
Right Wheel vel:  3.0591
Left Wheel vel:  2.6537


Fourth case: Right sensor nearest, objective angle -

In [10]:
odp=0.209947
tethad=-1.6546
S=-1
Lt_p=round((odp*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(((odp)*(v_max-v_min)+v_min),4)
print('vel: ',V_p)
W_R=round((1+((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

Ltp:  0.489
vel:  0.605
Right Wheel vel:  5.1024
Left Wheel vel:  1.1027


Fifth case: Front sensor nearest, objective angle +

In [11]:
odp=0.01
tethad=0.3
S=-1
Lt_p=round((odp*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(((odp)*(v_max-v_min)+v_min),4)
print('vel: ',V_p)
W_R=round((1+((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

Ltp:  0.309
vel:  0.505
Right Wheel vel:  2.1108
Left Wheel vel:  3.0687


Sixth case: Front sensor nearest, objective angle -

In [12]:
odp=0.01
tethad=-3
S=-1
Lt_p=round((odp*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(((odp)*(v_max-v_min)+v_min),4)
print('vel: ',V_p)
W_R=round((1+((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*tethad*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

Ltp:  0.309
vel:  0.505
Right Wheel vel:  7.3795
Left Wheel vel:  -2.2


### Dataset Creation

In [13]:
Df_C1['Min Value']=Df_C1[['LD','FD','RD']].min(axis=1)
Df_C1.sample(6)


,LD,FD,RD,HA,Min Value
1574,0.067886,0.031541,0.647589,2.6213,0.031541
254,0.704016,0.637563,0.800258,1.8408,0.637563
162,0.369158,0.322437,0.286307,2.7104,0.286307
2255,0.343719,0.339642,0.604664,2.2176,0.339642
1866,0.675004,0.880287,0.352840,3.0090,0.352840
1835,0.143776,0.948674,0.757905,1.3812,0.143776


### Left Wheel Evaluation

In [14]:
def lw_eval(df):
    if df['Min Value']== df['LD'] and df['HA']>0:
        Sf=-1
        odpf=df['LD']
    elif df['Min Value']== df['LD'] and df['HA']<0:
        Sf=0.5
        odpf=df['LD']
    elif df['Min Value']== df['RD'] and df['HA']>0:
        Sf=0.5
        odpf=df['RD']
    elif df['Min Value']== df['RD'] and df['HA']<0:
        Sf=-1
        odpf=df['RD']
    elif df['Min Value']== df['FD'] and df['HA']>0:
        Sf=-1
        odpf=df['FD']
    elif df['Min Value']== df['FD'] and df['HA']<0:
        Sf=-1
        odpf=df['FD']
    
        
      
    if odpf==0:
      odpf=0.001
      
    tethad=df['HA']
    
    if tethad>0 and tethad<0.3:
        tethad*=10
        
    if tethad<0 and tethad>-0.3:
        tethad=-0.3
    
    Lt_p=round((odpf*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(((odpf)*(v_max-v_min)+v_min),4)

    
    return round((1-((LMr*tethad*Sf)/(2*Lt_p)))*(V_p/R),4)

def rw_eval(df):
    if df['Min Value']== df['LD'] and df['HA']>0:
        Sf=-1
        odpf=df['LD']
    elif df['Min Value']== df['LD'] and df['HA']<0:
        Sf=0.5
        odpf=df['LD']
    elif df['Min Value']== df['RD'] and df['HA']>0:
        Sf=0.5
        odpf=df['RD']
    elif df['Min Value']== df['RD'] and df['HA']<0:
        Sf=-1
        odpf=df['RD']
    elif df['Min Value']== df['FD'] and df['HA']>0:
        Sf=-1
        odpf=df['FD']
    elif df['Min Value']== df['FD'] and df['HA']<0:
        Sf=-1
        odpf=df['FD']
      
      
    if odpf==0:
      odpf=0.001
      
    tethad=df['HA']
   
    if tethad>0 and tethad<0.3:
        tethad=0.3
        
    if tethad<0 and tethad>-0.3:
        tethad=-0.3
    
    Lt_p=round((odpf*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(((odpf)*(v_max-v_min)+v_min),4)

    
    return round((1+((LMr*tethad*Sf)/(2*Lt_p)))*(V_p/R),4)
        

Df_C1=Df_C1.assign(W_L=Df_C1.apply(lw_eval,axis=1))

Df_C1=Df_C1.assign(W_R=Df_C1.apply(rw_eval,axis=1))

Df_C1.sample(6)



        

,LD,FD,RD,HA,Min Value,W_L,W_R
1884,0.841218,0.266237,0.151667,0.9458,0.151667,2.3434,3.5622
2201,0.161212,0.137921,0.157390,0.3662,0.137921,3.3979,2.4380
4,0.202230,0.298065,0.772213,-1.4565,0.202230,3.9698,2.1953
1601,0.442332,0.120573,0.225497,0.1431,0.120573,4.7908,2.4713
1136,0.234100,0.434265,0.251538,-1.4730,0.234100,4.0340,2.2952
663,0.724453,0.829104,0.326656,-1.6869,0.326656,1.5613,5.2418


Extract WL to train 

In [15]:
Df_C1_Train=Df_C1[['LD','FD','RD','HA','W_L']]
Df_C1_Train.sample(6)

,LD,FD,RD,HA,W_L
2754,0.830928,0.317276,0.586636,-0.1398,3.0478
2776,0.451765,0.016631,0.797682,2.1636,6.0174
2660,0.664856,0.982652,0.403348,-0.3176,3.2701
930,0.365442,0.012186,0.631707,-2.4257,-1.2609
1618,0.100472,0.154265,0.384032,-1.7861,4.0511
1759,0.364871,0.442437,0.429961,2.5544,6.2094


In [16]:
Df_C1_Train.to_csv('Df_C1_Train_L_2.csv',index=False)

Extract WR to train 

In [17]:
Df_C1_TrainR=Df_C1[['LD','FD','RD','HA','W_R']]
Df_C1_TrainR.sample(6)

,LD,FD,RD,HA,W_R
2290,0.932400,0.441290,0.089283,0.2832,3.0026
102,0.267543,0.016631,0.400343,2.9098,-1.9804
2462,0.450908,0.995125,0.992417,-0.6381,3.4001
1507,0.710161,0.391685,0.026041,0.1571,2.8632
1976,0.444476,0.665090,0.129632,0.6115,3.3013
1176,0.859940,0.081720,0.159107,-1.0377,4.2420


In [18]:
Df_C1_TrainR.to_csv('Df_C1_Train_R_2.csv',index=False)

## No obstacle circle

### Angle Data Generation

In [19]:
HA_Data_2=[round(random.uniform(-math.pi, math.pi),4) for _ in range(3000)]
HA_Data_2=np.array(HA_Data).reshape(3000,1)
print('Angle data :',HA_Data_2[:2])

Angle data : [[-0.8852]
 [ 0.6788]]


In [20]:
Df_C2=pd.DataFrame(HA_Data_2,columns=[['HA']])
Df_C2.sample(6)

,HA
2802,-0.3352
1637,-1.9516
2238,0.4698
2842,-2.7153
1822,-1.8797
590,2.3653


### Evaluation   Right and Left Wheel

In [21]:
Lt_2=1.2
v_max=1

def lw_eval_NO(df):
    Sf1=-1
    haf=df['HA']
    Lt_pf=round((Lt_2),4)
    V_pf=round(v_max,4)
    return round((1-((LMr*haf*Sf1)/(2*Lt_pf)))*(V_pf/R),4)

def rw_eval_NO(df):
    Sf1=-1
    haf=df['HA']
    Lt_pf=round((Lt_2),4)
    V_pf=round(v_max,4)
    return round((1+((LMr*haf*Sf1)/(2*Lt_pf)))*(V_pf/R),4)


Df_C2=Df_C2.assign(W_L=Df_C2.apply(lw_eval_NO,axis=1))

Df_C2=Df_C2.assign(W_R=Df_C2.apply(rw_eval_NO,axis=1))


Df_C2.sample(6)



,HA,W_L,W_R
935,0.3304,5.3972,4.8592
2781,2.2059,6.9240,3.3324
416,2.0525,6.7992,3.4573
498,-0.0553,5.0832,5.1732
1997,1.3650,6.2395,4.0170
687,1.0109,5.9512,4.3052


Left Wheel Train NO

In [22]:
Df_C2_Train=Df_C2[['HA','W_L']]
Df_C2_Train.sample(6)

,HA,W_L
2571,-3.0514,2.6441
2000,2.9933,7.5651
780,-2.2501,3.2964
2375,-3.0593,2.6376
1762,3.0677,7.6256
2699,0.4179,5.4684


In [23]:
Df_C2_Train.to_csv('Df_C2_Train_L_2.csv',index=False)

Right Wheel Train NO

In [24]:
Df_C2_Train_R=Df_C2[['HA','W_R']]
Df_C2_Train_R.head(6)

,HA,W_R
0,-0.8852,5.8488
1,0.6788,4.5756
2,1.6664,3.7716
3,-2.0558,6.8018
4,-1.4565,6.3139
5,-2.3302,7.0252


In [25]:
Df_C2_Train_R.to_csv('Df_C2_Train_R_2.csv',index=False)